In [1]:
#bibliotecas
import numpy as np
import pandas as pd
import tabula
from zipfile import ZipFile

In [2]:
#titulo do arquivo pdf a ser lido
titulo_pdf = "Padrão_TISS_Componente_Organizacional_202103.pdf"

#titulo das tabelas
titulo_30 = 'Tabela de Tipo do Demandante'
titulo_31 = 'Tabela de Categoria do Padrão TISS'
titulo_32 = 'Tabela de Tipo de Solicitação'

#titulo do arquivo zip
titulo_zip = 'Teste_Intuitive_Care_Diogo_Filipe_Sens.zip'

In [3]:
#ler todas as tabelas do documento pdf
table = tabula.read_pdf(titulo_pdf, pages = "all")

In [4]:
#encontrar as tabelas solicitadas
for i in range(len(table)):
    if titulo_30 in table[i].columns.values:
        quadro_30 = pd.DataFrame(table[i])
    elif titulo_31 in table[i].columns.values:
        quadro_31 = pd.DataFrame(table[i])
        index = i #indice de referencia para apensar as subtabelas (ver adiante)
    elif titulo_32 in table[i].columns.values:
        quadro_32 = pd.DataFrame(table[i])


In [5]:
#criar um dataframe provisorio com rotulo da tabela 31
df_prov = pd.DataFrame(columns = table[index].columns.values)

for i in range(1,6):
    #adicionar o cabeçalho das subtabelas como linha do dataframe provisorio
    #obs: subtabelas sao as partes da tabela 31 que ficaram nas paginas seguintes
    df_prov[table[index].columns.values[0]] = [table[index + i].columns.values[0]]
    df_prov[table[index].columns.values[1]] = [table[index + i].columns.values[1]]

    #mudar o nome do rotulo das subtabelas para o rotulo da tabela 31
    table[index + i].columns = table[index].columns.values

    #apensar as subtabelas ao dataframe provisorio
    df_prov = df_prov.append(table[index + i])

    #apensar o dataframe provisorio a tabela 31
    quadro_31 = quadro_31.append(df_prov)

    #esvaziar o dataframe provisorio
    df_prov = pd.DataFrame(columns = table[index].columns.values)

In [6]:
#resetando os indices da tabela 31
quadro_31 = quadro_31.reset_index(drop=True)

In [7]:
#separando as colunas da tabela 30
quadro_30 = quadro_30.join(quadro_30[titulo_30].str.split(expand=True, n = 1))
quadro_30.columns.values[2] = quadro_30.columns.values[0]
quadro_30.columns.values[0] = 'Delete'
del quadro_30['Delete']

In [8]:
#separando as colunas da tabela 32
quadro_32 = quadro_32.join(quadro_32[titulo_32].str.split(expand=True, n = 1))
quadro_32.columns.values[2] = quadro_32.columns.values[0]
quadro_32.columns.values[0] = 'Delete'
del quadro_32['Delete']

In [9]:
#alterando o rotulo das colunas das tabelas
quadro_30.columns = [quadro_30[0][0], quadro_30[titulo_30][0]]
quadro_31.columns = [quadro_31['Unnamed: 0'][0], quadro_31[titulo_31][0]]
quadro_32.columns = [quadro_32[0][0], quadro_32[titulo_32][0]]

In [10]:
#eliminando a primeira linha das tabelas com informacao repetida
quadro_30 = quadro_30.loc[1:5]
quadro_31 = quadro_31.loc[1:131]
quadro_32 = quadro_32.loc[1:3]

In [11]:
#resetando os indices das tabelas
quadro_30 = quadro_30.reset_index(drop=True)
quadro_31 = quadro_31.reset_index(drop=True)
quadro_32 = quadro_32.reset_index(drop=True)

In [12]:
#salvando arquivos .csv
quadro_30.to_csv(titulo_30 + '.csv')
quadro_31.to_csv(titulo_31 + '.csv')
quadro_32.to_csv(titulo_32 + '.csv')

In [13]:
#criando arquivo zip dos arquivos csv
zipObj = ZipFile(titulo_zip, 'w')
zipObj.write(titulo_30 + '.csv')
zipObj.write(titulo_31 + '.csv')
zipObj.write(titulo_32 + '.csv')
zipObj.close()